The continued fraction algorithm applied to a real number $x_0 = x$ forms a sequence of partial quotients $a_n$ by the transformation
\begin{equation}
    a_n = \lfloor x_n \rfloor, \\
    x_{n+1} = \frac{1}{x_n - a_n}.
\end{equation}
The algorithm terminates if $x_n = a_n$; this happens if and only if the initial $x$ is rational. The convergents $P_n/Q_n$ are defined for $n > 0$ by
\begin{equation}
    P_n = a_nP_{n-1} + P_{n-2} \\
    Q_n = a_nQ_{n-1} + Q_{n-2}
\end{equation}
with initial conditions $P_{-2} = 0$, $P_{-1} = 1$ and $Q_{-2} = 1$, $Q_{-1} = 0$.

If $x = \sqrt{N}$ for some positive integer $N$, then each $x_n$ may be written in the form $(r + \sqrt{N})/s$ with $r$, $s$ integers and $s | (r^2 - N)$. We proceed by induction. For $n=0$, we have $x_0 = \sqrt{N}$. We can write this in the required form by choosing $r_0 = 0$ and $s_0 = 1$. Then
\begin{equation}
    x_0 = \frac{0 + \sqrt{N}}{1}
\end{equation}
where $r_0$ and $s_0$ are integers satisfying
\begin{equation}
    s_0 | (r_0^2 - N) \iff 1 | -N
\end{equation}
for any integer $N$.

Now assume that the statement is true for all $n \geq 0$. That is,
\begin{equation}
    x_n = \frac{r_n + \sqrt{N}}{s_n}
\end{equation}
where $r_n$ and $s_n$ are integers and $s_n | (r_n^2 - N)$.

The continued fraction algorithm defines
\begin{equation}
    a_n = \lfloor x_n \rfloor \quad \text{and} \quad x_{n+1} = \frac{1}{x_n - a_n}
\end{equation}
Substitute the expression for $x_n$ to get
\begin{equation}
    x_{n+1} = \frac{1}{\frac{r_n + \sqrt{N}}{s_n} - a_n} = \frac{1}{\frac{r_n + \sqrt{N} - a_n s_n}{s_n}} = \frac{s_n}{r_n - a_n s_n + \sqrt{N}}
\end{equation}
Now we rationalize the denominator
\begin{equation}
    x_{n+1} = \frac{s_n (a_n s_n - r_n + \sqrt{N})}{N - (r_n - a_n s_n)^2}
\end{equation}
From this expression, we define
\begin{equation}
    r_{n+1} = a_n s_n - r_n, \\
    s_{n+1} = \frac{N - (r_n - a_n s_n)^2}{s_n} = \frac{N - r_{n+1}^2}{s_n}.
\end{equation}
Our expression for $x_{n+1}$ is now in the correct form
\begin{equation}
    x_{n+1} = \frac{s_n (r_{n+1} + \sqrt{N})}{s_n s_{n+1}} = \frac{r_{n+1} + \sqrt{N}}{s_{n+1}}
\end{equation}
By definition, $a_n = \lfloor x_n \rfloor$ is an integer and by the inductive hypothesis, $r_n$ and $s_n$ are integers. Therefore, $r_{n+1} = a_ns_n - r_n$ is an integer. Expanding $s_{n+1}$,
\begin{equation}
    s_{n+1} = \frac{N - (r_n^2 - 2a_n r_n s_n + a_n^2 s_n^2)}{s_n} = \frac{N - r_n^2}{s_n} + 2a_n r_n - a_n^2 s_n
\end{equation}
By the inductive hypothesis, $s_n | (r_n^2 - N)$, which means that $(r_n^2 - N) / s_n$ is an integer. Since $a_n, r_n, s_n$ are all integers, the entire expression for $s_{n+1}$ is an integer. From our definition, $s_{n+1} = (N - r_{n+1}^2) / s_n$, which rearranges to $r_{n+1}^2 - N = -s_n s_{n+1}$. Since $s_n$ is an integer, this shows that $s_{n+1} | (r_{n+1}^2 - N)$.

The recurrence relations in the proof allow us to create an algorithm using only integer arithmetic. Using $a_0 = \lfloor \sqrt{N} \rfloor$,
\begin{align}
    r_{n+1} &= a_ns_n - r_n, \\
    s_{n+1} &= \frac{N - r_{n+1}^2}{s_n}, \\
    a_{n+1} &= \left\lfloor \frac{r_{n+1} + \sqrt{N}}{s_{n+1}} \right\rfloor \approx \left\lfloor \frac{r_{n+1} + a_0}{s_{n+1}} \right\rfloor.
\end{align}

In [40]:
import math

def get_continued_fraction_sqrt(N):
    '''
    Calculates the continued fraction expansion of sqrt(N) using integer arithmetic.
    Also returns the maximum values of the intermediate r and s coefficients.
    '''
    if N < 0:
        raise ValueError("Input must be a non-negative integer.")

    m0 = math.isqrt(N)
    # Handle perfect squares, which have a finite expansion.
    if m0 * m0 == N:
        return ([m0], []), 0, 1

    # Initialise the algorithm based on the proof
    a0 = m0
    r, s = 0, 1
    period = []

    # Track maximum values of r and s
    max_r, max_s = 0, 1

    # The period for sqrt(N) is known to start right after a0 and
    # terminates when a term equal to 2*a0 is found.
    while True:
        # Calculate the next terms using the integer recurrence relations
        r = a0 * s - r
        s = (N - r * r) // s
        if s == 0:
            break

        a0 = (m0 + r) // s
        period.append(a0)

        # Update maximums
        max_r = max(max_r, r)
        max_s = max(max_s, s)

        # The period terminates when a_k = 2 * m0
        if a0 == 2 * m0:
            break

    return ([m0], period), max_r, max_s

print("Continued Fractions for sqrt(N) for 1 <= N <= 50")
print("-" * 100)
header = f"{'N':<4}| {'Continued Fraction':<45}| {'Max r':<8}| {'Max s':<8}| {'Bounds (sqrt(N), 2*sqrt(N))'}"
print(header)
print("-" * 100)

for n in range(1, 51):
    (non_periodic, periodic), r_max, s_max = get_continued_fraction_sqrt(n)

    # Format the output string for the continued fraction
    if not periodic: # Perfect square
        cf_str = f"{non_periodic}"
    else:
        cf_str = f"{non_periodic[0]}; {periodic}"

    # Format the bounds
    sqrt_n = math.sqrt(n)
    bounds_str = f"({sqrt_n:.2f}, {2*sqrt_n:.2f})"

    print(f"{n:<4}| {cf_str:<45}| {r_max:<8}| {s_max:<8}| {bounds_str}")

Continued Fractions for sqrt(N) for 1 <= N <= 50
----------------------------------------------------------------------------------------------------
N   | Continued Fraction                           | Max r   | Max s   | Bounds (sqrt(N), 2*sqrt(N))
----------------------------------------------------------------------------------------------------
1   | [1]                                          | 0       | 1       | (1.00, 2.00)
2   | 1; [2]                                       | 1       | 1       | (1.41, 2.83)
3   | 1; [1, 2]                                    | 1       | 2       | (1.73, 3.46)
4   | [2]                                          | 0       | 1       | (2.00, 4.00)
5   | 2; [4]                                       | 2       | 1       | (2.24, 4.47)
6   | 2; [2, 4]                                    | 2       | 2       | (2.45, 4.90)
7   | 2; [1, 1, 1, 4]                              | 2       | 3       | (2.65, 5.29)
8   | 2; [1, 4]                               

For any integer $N$ that is not a perfect square, the continued fraction of $\sqrt{N}$ is periodic. The periodic part starts immediately after the first term and is a palindrome. The last term of the periodic part is always equal to the initial term $a_k = 2a_0$ which we use to detect the end of the period.

It is a known theoretical result that for the continued fraction of $\sqrt{N})$, the intermediate integers have bounds $r_n < \sqrt{N}$ and $s_n < 2\sqrt{N}$. Thus, the maximum value that $r$ can reach is $\lfloor\sqrt{N}\rfloor$, and the maximum value of $s$ approaches, but does not exceed, $2\sqrt{N}$. This property is essential to make our algorithm stable and efficient by preventing unbounded growth of integers.

For a fixed value of the positive integer $N$, the equation $x^2 - Ny^2 = 1$, in integer unknowns $x$ and $y$, is called Pell's equation. The negative Pell equation is $x^2 - Ny^2 = -1$. The convergents $P_n/Q_n$ of the continued fraction of $\sqrt{N}$ provide good rational approximations. The values of $P_n^2 - NQ_n^2$ are always small integers that eventually hit $1$ or $-1$. The fundamental solution to Pell's equation or its negative form is always found among these convergents. In particular, if the period length of the continued fraction of $\sqrt{N}$ is $k$, then the fundamental solution is found at the
$(k-1)$th or $(2k-1)$th convergent.

We claim that the negative Pell equation is insoluble if $N$ has any prime factor $p$ of the form $4k + 3$.

Assume a solution $(x, y)$ exists the negative Pell equation such that $p \equiv 3 \mod{4}$ is a prime factor of $N$. Then taking modulo $p$,
\begin{equation}
    x^2 \equiv -1 \mod{p},
\end{equation}
in other words, $-1$ is a quadratic residue modulo $p$. This contradicts the law of quadratic reciprocity which states that $p \equiv 1 \mod{4}$.

When $x$, $y$ or $N$ are large, calculating $x^2$ or $Ny^2$ is computationally expensive. We can use modular arithmetic to test the equation
\begin{equation}
    x^2 - Ny^2 = \pm 1
\end{equation}
probabilistically but with high confidence. If the equation holds, then it must also hold modulo any prime $p$. If it fails for even one prime, it cannot be true.

*   Select a set of sufficiently large, distinct prime numbers $p_1, \dots, p_k$ as to not be special factors of $x$, $y$, or $N$.
*   For each prime p in the set:
    *   Calculate the remainders $x' = x \mod{p}$, $y' = y \mod{p}$ and $N' = N \mod{p}$.
    *   Compute $x'^2 - N'y'^2 \mod{p}$, taking ample modulo $p$ steps to prevent overflow.
    *   Check if this is congruent to $1$ or $-1$ modulo $p$.
    *   If is not equal to $1$ and not equal to $p-1$, then the equation is false and we terminate the proceedure.
*   If the check passes for all selected primes, then the equation is likely to be true.

In [41]:
def check_pell_large(x, y, N):
    '''
    Tests if x^2 - N*y^2 = +/-1 for large integers
    '''
    if x == 0 or y == 0 or N == 0:
        return False

    # A set of large, randomly chosen primes
    primes = [10**9 + 7, 10**9 + 9, 10**9 + 21, 10**9 + 33, 10**9 + 87]

    for p in primes:
        try:
            # Calculate (x^2 - N*y^2) mod p
            # pow(base, exp, mod) is efficient for modular exponentiation
            lhs = (pow(x, 2, p) - (N % p) * pow(y, 2, p)) % p
        except OverflowError:
            print("Error: Inputs are too large for standard types.")
            return False

        # Check if the result is congruent to 1 or -1 (p-1)
        if lhs != 1 and lhs != p - 1:
            return False

    # If the check passes for all primes, the equation is very likely true.
    return True

x_61 = 1766319049
y_61 = 226153980
N_61 = 61
print(f"Checking solution for N=61: {check_pell_large(x_61, y_61, N_61)}")
print(f"Checking a false case: {check_pell_large(123, 456, 789)}")

Checking solution for N=61: True
Checking a false case: False


In [42]:
def solve_pell_equation(N):
    '''
    Finds the fundamental solution (x, y) to Pell's equation x^2 - N*y^2 = 1
    using the continued fraction method for sqrt(N).
    Returns (x, y) or None if N is a perfect square.
    '''
    # Pell's equation is trivial for perfect squares.
    if math.isqrt(N)**2 == N:
        return None

    # Initial conditions for convergents P_n/Q_n
    # P-2=0, P-1=1, Q-2=1, Q-1=0
    p_prev, p_curr = 0, 1
    q_prev, q_curr = 1, 0

    # Initial conditions for the continued fraction of sqrt(N)
    a0 = m = math.isqrt(N)
    d = 1

    # The first convergent
    p_curr, p_prev = a0 * p_curr + p_prev, p_curr
    q_curr, q_prev = a0 * q_curr + q_prev, q_curr

    # Loop until we find a solution
    while True:
        # Calculate next terms of the continued fraction
        d = (N - m*m) // d
        a = (a0 + m) // d
        m = a * d - m

        # Calculate the next convergent (p, q)
        p_curr, p_prev = a * p_curr + p_prev, p_curr
        q_curr, q_prev = a * q_curr + q_prev, q_curr

        # Check if this convergent is the solution
        if p_curr**2 - N * q_curr**2 == 1:
            return p_curr, q_curr

def tabulate_pell_solutions(ranges):
    '''
    Tabulates the fundamental solution to Pell's equation for N in given ranges.
    '''
    print(f"{'N':<5} | {'x':<40} | {'y'}")
    print("-" * 60)
    for start, end in ranges:
        print(f"\n--- Range: {start} to {end} ---\n")
        for n in range(start, end + 1):
            solution = solve_pell_equation(n)
            if solution:
                x, y = solution
                print(f"{n:<5} | {x:<40} | {y}")
            else:
                print(f"{n:<5} | {'(Perfect square)':<40} |")

ranges_to_solve = [
    (1, 100),
    (500, 550)
]
print("Fundamental solutions (x,y) to Pell's equation x^2 - Ny^2 = 1")
tabulate_pell_solutions(ranges_to_solve)

Fundamental solutions (x,y) to Pell's equation x^2 - Ny^2 = 1
N     | x                                        | y
------------------------------------------------------------

--- Range: 1 to 100 ---

1     | (Perfect square)                         |
2     | 3                                        | 2
3     | 2                                        | 1
4     | (Perfect square)                         |
5     | 9                                        | 4
6     | 5                                        | 2
7     | 8                                        | 3
8     | 3                                        | 1
9     | (Perfect square)                         |
10    | 19                                       | 6
11    | 10                                       | 3
12    | 7                                        | 2
13    | 649                                      | 180
14    | 15                                       | 4
15    | 4                                        | 1
16    |